***Imports***

In [1]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import subprocess


***Ollama LLM Wrapper (phi3:mini)***

In [2]:
def ollama_llm(prompt, timeout=90):
    try:
        process = subprocess.Popen(
            ["ollama", "run", "phi3:mini"],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        output, error = process.communicate(
            input=prompt,
            timeout=timeout
        )

        if process.returncode != 0:
            return f"Error: {error.strip()}"

        return output.strip()

    except subprocess.TimeoutExpired:
        process.kill()
        return "Error: Ollama took too long and was terminated."



***Distance Function***

In [3]:
geolocator = Nominatim(user_agent="tripgenix", timeout=10)

def calculate_distance(source, destination):
    src = geolocator.geocode(source)
    dest = geolocator.geocode(destination)

    if not src or not dest:
        return None

    return round(
        geodesic(
            (src.latitude, src.longitude),
            (dest.latitude, dest.longitude)
        ).km, 2
    )


***Transport Options***

In [4]:
def transport_options(distance_km):
    return {
        "Bus": {"cost": round(distance_km * 2), "time_hr": round(distance_km / 40, 1)},
        "Train": {"cost": round(distance_km * 1.5), "time_hr": round(distance_km / 60, 1)},
        "Flight": {"cost": round(distance_km * 6), "time_hr": round(distance_km / 700, 1)}
    }


***Best Transport (LLM Reasoning)***

In [5]:
def full_travel_plan(source, destination, distance, options, budget):
    prompt = f"""
You are a smart travel planner.

Source: {source}
Destination: {destination}
Distance: {distance} km
Budget: ₹{budget}

Transport options:
{options}

Give:
1. Best transport under budget
2. Tourist places
3. Food
4. Stay options
"""
    return ollama_llm(prompt)


In [6]:
source = "Delhi"
destination = "Jaipur"
budget = 5000


In [7]:
distance = calculate_distance(source, destination)

if distance is None:
    raise ValueError("Location not found")

options = transport_options(distance)


***Tourist Places***

In [8]:
print("Distance:", distance, "km")
print("\nGenerating travel plan...\n")

print(
    full_travel_plan(
        source, destination, distance, options, budget
    )
)


Distance: 232.8 km

Generating travel plan...

Error: Ollama took too long and was terminated.


***Food & Stay***

***Test Cell (MOST IMPORTANT)***